In [ ]:
import numpy as np
import sys
np.set_printoptions(precision=10, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)

In [ ]:
# options for mgf
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "6-31G",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}


psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)


In [ ]:
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "6-31G",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'cas',
    'nact_orbs' : 11, 
    'nact_els' : 4,
    'full_diagonalization' : True,
    'number_of_photons' : 0
}

# Need to determine appropriate omega later~
#cavity_dict = {
#    'omega_value' : 0.0,
#    'lambda_vector' : np.array([0, 0, 0.0]),
#    'ci_level' : 'cas',
#    'nact_orbs' : 11, 
#    'nact_els' : 4,
#    'full_diagonalization' : True,
#    'number_of_photons' : 0
#}

mol = psi4.geometry(mol_str)
psi4.set_options(options_dict)

test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_free_dict
)



In [ ]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 50
all_states = np.linspace(0, N_el-1, N_el, dtype=int)
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))
Mu_array = np.zeros((N_R, N_el, N_el, 3))
psi4.set_options(options_dict)

r_idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    print(mol_str)
    mol = psi4.geometry(mol_str)
    #scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    #fci_energy = psi4.energy('fci',ref_wfn=wfn)
    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    #assert np.isclose(fci_energy, LiH.CIeigs[0], 1e-7)
    # sort out states with dipole-allowed transition from ground state
    #singlet_states = LiH.sort_dipole_allowed_states( N_el)
    #print(singlet_states)
    
    #E_array[r_idx, :] = LiH.CIeigs[singlet_states]
    #Mu_array[r_idx, :, :, :] = LiH.compute_dipole_moments(singlet_states)
    
    # uncomment to try not sorting singlet states
    E_array[r_idx, :] = LiH.CIeigs[all_states]
    Mu_array[r_idx, :, :, :] = LiH.compute_dipole_moments(all_states)
    
    r_idx += 1



In [ ]:
from matplotlib import pyplot as plt


In [ ]:
print(r_array)
print(E_array[:,1]-E_array[:,0])

plt.plot(r_array, E_array[:,0]+0.1208, label="E0")
plt.plot(r_array, E_array[:,1], label="E1")
plt.plot(r_array, E_array[:,2], label="E2")
plt.plot(r_array, E_array[:,3], label="E3")
plt.legend()
#plt.xlim(1.79, 1.81)
plt.show()

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
def build_pf_hamiltonian(n_el, n_ph, E_R, omega, lamvec, mu):
    """
    Given an array of n_el E_R values and an n_ph states with fundamental energy omega
    build the PF Hamiltonian
    
    n_el : int
        the number of electronic states (n_el = 1 means only ground-state)
    
    n_ph : int
        the number of photon occupation states (n_ph = 1 means only the |0> state)
    
    E_R : np.array of floats
        the electronic energies
        
    omega : float
        the energy of the photonic mode
        
    lamvec : np.array of floats
        the lambda vector
        
    mu : (n_el x n_el x 3) np.array of floats 
        mu[i, j, k] is the kth cartesian component of the dipole moment expectation value between 
        state i and state j
    
    """
    H_PF = np.zeros((n_el * n_ph, n_el * n_ph))
    
    # take care of the diagonals first
    # bare electronic and photonic energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            H_PF[na,na] = E_R[a] + n * omega
        
    # diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for g in range(n_el):
                H_PF[na,na] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g,a,:])
            
    # off-diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for b in range(n_el):
                nb = n * n_el + b
                for g in range(n_el):
                    if a != b:
                        H_PF[na, nb] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g, b, :])
                
    # off-diagonal bilinear coupling
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            
            for m in range(n_ph):
                for b in range(n_el):
                    mb = m * n_el + b
                    
                    if n == (m-1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na,mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        
                    elif n == (m+1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na, mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1)
                        
    
    return H_PF
                
            
    
        
    
    

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
np.set_printoptions(precision=6, linewidth=200, suppress=True)
omega_cav = 0.12086
lambda_vector = np.array([0., 0., 0.08])
N_el = 3
N_ph = 2

_HPF = build_pf_hamiltonian(N_el, N_ph, E_array[0,:], omega_cav, lambda_vector, Mu_array[0, :, :, :])
print("H")
print(_HPF)

We want to investigate the mean absolute error of a given polariton potential energy surface as a 
function of electronic and photonic truncation:

$$ {\rm MAE} = \frac{1}{N_R} \sum_i^{N_R} \left| E(R_i) - E^{'}(R_i) \right| $$

where $E^{'}(R_i)$ is a value of the polaritonic PES for a truncated level of theory and $E(R_i)$ is the exact value.

In [ ]:
def MAE(E, Ep):
    """
    function to return the mean absolute error of the approximate PES stored in Ep relative to 
    the exact PES stored in E
    """
    # get length
    _NR = len(E)
    
    # take element-by-element difference
    _diff = E - Ep
    
    # compute absolute value of differences
    _absdiff = np.abs(_diff)
    
    # return sum divided by _NR
    return np.sum(_absdiff) / _NR

    

In [ ]:
N_el = 50
N_ph = 2

#PF_EIG = np.zeros((N_R, N_el * N_ph))
#PF_5050 = np.zeros((N_R, N_el * N_ph))
MAE_array = np.zeros((20,20,3))
N_el_array = np.linspace(2, 21, 20, dtype=int)
N_ph_array = np.linspace(2, 21, 20, dtype=int)
print(N_el_array)
print(N_ph_array)

In [ ]:

N_el = 10
N_ph = 10
pf_array = np.zeros((N_R, N_el * N_ph ))
for k in range(N_R):           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)
    pf_array[k,:] = pf_e



#print(r_array[i], pf_e[0], pf_e[1],  pf_e[2], pf_e[2])
#mae_g = MAE(pf_tmp[:,0], PF_5050[:,0])
#mae_lp = MAE(pf_tmp[:,1], PF_5050[:,1])
#mae_up = MAE(pf_tmp[:,2], PF_5050[:,2])
#MAE_array[i, j, 0] = mae_g
#MAE_array[i, j, 1] = mae_lp
#MAE_array[i, j, 2] = mae_up
#print(F'Nel : {N_el}, Nph : {N_ph}, mae_g : {mae_g}, mae_lp : {mae_lp}, mae_up : {mae_up}')


In [ ]:
#plt.plot(r_array, Eg_QED_FCI_Np_1)
plt.plot(r_array, pf_array[:,0])
plt.plot(r_array, pf_array[:,1])
#plt.plot(r_array, E3_QED_FCI_Np_1, label="QED-FCI")
#plt.plot(r_array, E3_QED_FCI_Np_5, label="QED-FCI")
plt.plot(r_array, pf_array[:,2])
plt.plot(r_array, pf_array[:,3])
#plt.plot(r_array, pf_array[:,4])
plt.legend()

In [ ]:
for i in range(N_R):
    print(F'{r_array[i]}, {pf_array[i,0]}, {pf_array[i,1]}, {pf_array[i,2]}, {pf_array[i,3]}, {pf_array[i,3]} ')
      

In [ ]:
# we will compare to this!
#PF_2020 = np.copy(PF_EIG)
#PF_22 = np.copy(PF_EIG)

print(MAE(PF_5020[:,0], PF_2020[:,0]))
print(MAE(PF_5020[:,1], PF_2020[:,1]))
print(MAE(PF_5020[:,2], PF_2020[:,2]))

print(MAE(PF_5050[:,0], PF_2020[:,0]))
print(MAE(PF_5050[:,1], PF_2020[:,1]))
print(MAE(PF_5050[:,2], PF_2020[:,2]))

print(MAE(PF_5050[:,0], PF_5020[:,0]))
print(MAE(PF_5050[:,1], PF_5020[:,1]))
print(MAE(PF_5050[:,2], PF_5020[:,2]))
#omega_cav = 0.120000
#lambda_vector = np.array([0, 0.0, 0.05])
#numpy.savetxt(fname, X, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)[source]
np.savetxt('PF_5050_om_zp12_lamz_zp05.out', PF_5050, fmt='%16.12e', delimiter=',', newline='\n', header="N_el = 50, N_ph = 50, omega = 0.12, lambda = 0., 0., 0.05, r_array = np.linspace(1.4, 2.2, N_R)" ) 


In [ ]:
for i in range(20):

    for j in range(20):
        N_el = N_el_array[i]
        N_ph = N_ph_array[j]
        print(F'{N_el}, {N_ph}, {MAE_array[i, j, 0]}, {MAE_array[i, j, 1]}, {MAE_array[i, j, 2]}')
        
    print('')


In [ ]:
#plt.plot(r_array, PF_2020[:,0], 'black', label="LP - 20-20")
#plt.plot(r_array, PF_WTRIP[:,0], 'ro', label="LP - 2-2")
plt.plot(r_array, PF_2020[:,1], 'r--', label="LP - 20-2")
plt.plot(r_array, PF_WTRIP[:,2], 'purple', label="UP - 20-20")
plt.plot(r_array, PF_2020[:,2], 'b--', label="UP - 20-2")
plt.plot(r_array, PF_WTRIP[:,3], 'bo', label="UP - 2-2")
plt.legend()
plt.show()


plt.plot(r_array,PF_2020[:,2]- PF_WTRIP[:,3])
#plt.xlim(1.62, 2.2)
plt.show()

print(PF_2020[:,2]- PF_WTRIP[:,3])

In [ ]:
plt.plot(r_array, PF_2020[:,1], 'black', label="LP - 20-20")
plt.plot(r_array, PF_22[:,1], 'ro', label="LP - 2-2")
plt.plot(r_array, PF_202[:,1], 'r--', label="LP - 20-2")
plt.plot(r_array, PF_2020[:,2], 'purple', label="UP - 20-20")
plt.plot(r_array, PF_202[:,2], 'b--', label="UP - 20-2")
plt.plot(r_array, PF_22[:,2], 'bo', label="UP - 2-2")
plt.legend()
plt.show()

#plt.plot(r_array, PF_EIG[:,2], 'b--')
#plt.plot(r_array, E_array[:,0]+omega_cav)
#plt.plot(r_array, E_array[:,2])
#plt.plot(r_array, E_array[:,3])
#plt.plot(r_array, PF_EIG[:,3])
#plt.plot(r_array, PF_EIG[:,4])
#plt.plot(r_array, PF_EIG[:,5])





In [ ]:

mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}

cavity_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 0
}



E_r_1p8_array = np.zeros(N_el)
Mu_r_1p8_array = np.zeros((N_el, N_el, 3))



mol = psi4.geometry(mol_str)
scf_e, wfn = psi4.energy('SCF', return_wfn=True)
fci_energy = psi4.energy('fci',ref_wfn=wfn)

LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
assert np.isclose(fci_energy, LiH.CIeigs[0], 1e-7)
# sort out states with dipole-allowed transition from ground state
singlet_states = LiH.sort_dipole_allowed_states( N_el)
E_r_1p8_array[:] = LiH.CIeigs[singlet_states]
Mu_r_1p8_array = LiH.compute_dipole_moments(singlet_states)
    



In [ ]:
lam_z = np.linspace(0, 0.1, 50)
pol_v_l = np.zeros((50,3))
print(lam_z)

In [ ]:
N_el = 20
N_ph = 20
for i in range(50):
    lambda_vector[2] = lam_z[i]           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)
    pol_v_l[i,:] = pf_e[:3]
    print(lam_z[i], pf_e[0], pf_e[1], pf_e[2])

In [ ]:
plt.plot(lam_z, pol_v_l[:,0], label='g')
plt.plot(lam_z, pol_v_l[:,1], label='LP')
plt.plot(lam_z, pol_v_l[:,2], label='UP')

plt.legend()
plt.show()